# 04 – Evaluation & Analysis

Loads the trained models, makes predictions on the held-out test set, computes **accuracy, precision, recall, F1, MCC**, draws confusion matrices, bar-plots and saves a final `results.pkl` + CSV summary.

**Inputs**  
- `qvc_model.pkl`  
- `qsvm_model.pkl`  
- `preprocessing_objects.pkl`  
- `X_selected.csv` & `y.csv` (for test split)

**Outputs**  
- `model_comparison.csv`  
- `results.pkl`  
- PNG visualisations

In [ ]:
!pip install -q pandas numpy scikit-learn matplotlib seaborn

In [ ]:
import pandas as pd, numpy as np, pickle, matplotlib.pyplot as plt, seaborn as sns
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, matthews_corrcoef, confusion_matrix,
                             classification_report)
from datetime import datetime
print(f"Start: {datetime.now():%Y-%m-%d %H:%M:%S}")

## 1. Load artefacts

In [ ]:
base = '/content/drive/MyDrive/QuantumBoost2025/'
with open(base+'qvc_model.pkl','rb') as f:   qvc = pickle.load(f)
with open(base+'qsvm_model.pkl','rb') as f:  qsvm = pickle.load(f)
with open(base+'preprocessing_objects.pkl','rb') as f: prep = pickle.load(f)

X = pd.read_csv(base+'X_selected.csv')
y = pd.read_csv(base+'y.csv').squeeze()

# Re-create test set (same split as training notebook)
from sklearn.model_selection import train_test_split
_, X_test, _, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Apply same preprocessing pipeline
X_test_r = prep['robust'].transform(X_test)
X_test_q = prep['minmax'].transform(X_test_r)
X_test_pca = prep['pca'].transform(X_test_q)
X_test_pca = prep['pca_scaler'].transform(X_test_pca)

## 2. Predictions

In [ ]:
y_pred_qvc  = qvc.predict(X_test_pca)
y_pred_qsvm = qsvm.predict(X_test_pca)

## 3. Metrics helper

In [ ]:
def metrics(y_true, y_pred, name):
    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred, zero_division=0),
        'recall': recall_score(y_true, y_pred, zero_division=0),
        'f1': f1_score(y_true, y_pred, zero_division=0),
        'mcc': matthews_corrcoef(y_true, y_pred)
    }

## 4. Compute & display

In [ ]:
qvc_met  = metrics(y_test, y_pred_qvc,  "QVC")
qsvm_met = metrics(y_test, y_pred_qsvm, "QSVM")

comp = pd.DataFrame([['QVC']+list(qvc_met.values()),
                     ['QSVM']+list(qsvm_met.values())],
                    columns=['Model','Accuracy','Precision','Recall','F1','MCC'])
print(comp)

best = comp.loc[comp['MCC'].idxmax(),'Model']
print(f"\nBest model (MCC): {best}")

## 5. Visualisations

In [ ]:
# Bar plot
fig, ax = plt.subplots(1,5,figsize=(20,4))
for i, col in enumerate(comp.columns[1:]):
    comp.plot(x='Model', y=col, kind='bar', ax=ax[i], legend=False, color=['#1f77b4','#ff7f0e'])
    ax[i].set_ylim(0,1)
    ax[i].set_title(col)
plt.tight_layout()
plt.savefig(base+'model_comparison.png', dpi=300)
plt.show()

# Confusion matrices
cm_qvc  = confusion_matrix(y_test, y_pred_qvc)
cm_qsvm = confusion_matrix(y_test, y_pred_qsvm)

fig, ax = plt.subplots(1,2,figsize=(10,4))
sns.heatmap(cm_qvc, annot=True, fmt='d', cmap='Blues', ax=ax[0])
ax[0].set_title('QVC')
sns.heatmap(cm_qsvm, annot=True, fmt='d', cmap='Oranges', ax=ax[1])
ax[1].set_title('QSVM')
plt.savefig(base+'confusion_matrices.png', dpi=300)
plt.show()

## 6. Save final results

In [ ]:
comp.to_csv(base+'model_comparison.csv', index=False)
with open(base+'results.pkl','wb') as f:
    pickle.dump({'qvc':qvc_met, 'qsvm':qsvm_met, 'comparison':comp, 'best':best}, f)
print("Evaluation complete – all files saved.")